# Практика 4

На этой практике мы попробуем описанную в лекции LSTM - это не очень приятно, но мы справимся :)

p.s. рекомендуем на этой практике включить ГПУ - это поможет. Для этого жмем "среда выполнения" -> сменить среду выполнения, выбираем там T4 GPU

Для начала делаем что? Копируем гит:

In [ ]:
!git clone https://github.com/ArChanDDD/TS-MCSSirius-2024.git

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

## LSTM

Если вдруг захочется почитать побольше - есть класная статейка на [хабре](https://habr.com/ru/companies/wunderfund/articles/331310/)

In [ ]:
df = pd.read_csv('/content/TS-MCSSirius-2024/data/DailyDelhiClimateTrain.csv')
df['date'] = [datetime.strptime(x, '%Y-%m-%d') for x in list(df['date'])]
df.head()

In [ ]:
import matplotlib.pyplot as plt

# Посмотрим че у нас уже есть
train_df = df[:-130]
test_df = df[-130:]
_, _ = plt.subplots(1, 1, figsize=(20,10))
_ = plt.plot(train_df['date'], train_df['meantemp'])
_ = plt.plot(test_df['date'], test_df['meantemp'])

### Подготовка датасета

В PyTorch операции происходят не с обычными листами или массивами, а с особыми - их называют Тензоры.

Наша задача - сделать

In [ ]:
import torch

def create_dataset(dataset, lookback, is_test=False):
    X, y = [], []
    if not is_test:
        for i in range(len(dataset)-lookback):
            feature = dataset[i:i+lookback]
            target = dataset[i+1:i+lookback+1]
            X.append(feature)
            y.append(target)
    else:
        for i in range(len(train_df) - lookback, len(dataset)-lookback):
              feature = dataset[i:i+lookback]
              target = dataset[i+1:i+lookback+1]
              X.append(feature)
              y.append(target)
    return torch.tensor(X), torch.tensor(y)

In [ ]:
lookback = 20

X_train, y_train = create_dataset(list(train_df['meantemp']), lookback=lookback)
X_test, y_test = create_dataset(list(df['meantemp']), lookback=lookback, is_test=True)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

### Определим модель

In [ ]:
import torch.nn as nn

class AirModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=lookback, hidden_size=64, num_layers=4, batch_first=True)
        self.linear = nn.Linear(64, 1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

### И обучим ее!

In [ ]:
import numpy as np
import torch.optim as optim
import torch.utils.data as data

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AirModel().to(device)
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=32)

n_epochs = 1000
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    if epoch % 100 != 0:
        continue
    model.eval()
    with torch.no_grad():
        y_pred = model(X_train)
        train_rmse = np.sqrt(loss_fn(y_pred.cpu(), y_train.cpu()))
        y_pred = model(X_test)
        test_rmse = np.sqrt(loss_fn(y_pred.cpu(), y_test.cpu()))
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

### Не совсем честное предсказание

In [ ]:
forecast_len = len(test_df)

test_inputs = train_df['meantemp'][-lookback:].tolist()
print(test_inputs)

In [ ]:
model.eval()

seq = X_train

for i in range(forecast_len):
    #seq = torch.vstack((seq, torch.Tensor(test_inputs[-lookback:]).reshape(1, -1).to(device)))
    seq = torch.vstack((seq, X_test[i]))
    with torch.no_grad():
        test_inputs.append(model(seq)[-1].item())

forecasted = test_inputs[lookback:]

In [ ]:
def plot_pred(y_pred):
  _, _ = plt.subplots(1, 1, figsize=(20,10))
  _ = plt.plot(train_df['date'], train_df['meantemp'], label='train')
  _ = plt.plot(test_df['date'], test_df['meantemp'], label='val')
  _ = plt.plot(test_df['date'], list(y_pred), label='pred')
  _ = plt.legend()

In [ ]:
plot_pred(forecasted)

In [ ]:
def mse(y_pred):
  return np.mean((np.array(test_df['meantemp']) - np.array(y_pred)) ** 2)

In [ ]:
mse(forecasted)

### А теперь честное

In [ ]:
forecast_len = len(test_df)

test_inputs = train_df['meantemp'][-lookback:].tolist()
print(test_inputs)

In [ ]:
model.eval()

seq = X_train

for i in range(forecast_len):
    seq = torch.vstack((seq, torch.Tensor(test_inputs[-lookback:]).reshape(1, -1).to(device)))
    with torch.no_grad():
        test_inputs.append(model(seq)[-1].item())

forecasted = test_inputs[lookback:]

In [ ]:
def plot_pred(y_pred):
  _, _ = plt.subplots(1, 1, figsize=(20,10))
  _ = plt.plot(train_df['date'], train_df['meantemp'], label='train')
  _ = plt.plot(test_df['date'], test_df['meantemp'], label='val')
  _ = plt.plot(test_df['date'], list(y_pred), label='pred')
  _ = plt.legend()

In [ ]:
plot_pred(forecasted)

In [ ]:
mse(forecasted)

Что с этим делать - вам уже решать. Можно докрутить модельку, чтобы она предсказывала на генерируемых данных чуть лучше. А можно переписать процесс генерации и получить выход чуть получше :)